#carregar o dataset Lista_valvulas.xlsx

In [ ]:


import pandas as pd

# Caminho do arquivo
file_path = "Lista_valvulas.xlsx"

# Carregar a planilha, ignorando a primeira linha (header=1)
#xls = pd.excelFile(file_path, engine="openpyxl")
df = pd.read_excel(xls, sheet_name=xls.sheet_names[0], header=1, engine="openpyxl")

# Exibir informações básicas
print("Linhas:", len(df))
print("Colunas:", len(df.columns))
print("Nome das colunas:", df.columns.tolist()[:10])  # Mostra as 10 primeiras colunas
print(df.head())  # Visualiza as primeiras linhas

#Padronizar nomes de colunas

In [ ]:


# ------------------------------------------------------------------------------------
# PASSO 1: Padronização dos nomes de colunas
# - Coloca tudo em MAIÚSCULAS
# - Remove quebras de linha e espaços duplicados
# - Facilita o uso consistente dos campos ao longo do pipeline
# ------------------------------------------------------------------------------------

# Se o seu DataFrame já se chama df (carregado com header=1), aplique:
df.columns = (
    df.columns
      .str.strip()               # remove espaços no início/fim
      .str.replace("\n", " ")    # substitui quebras de linha por espaço
      .str.replace("  ", " ")    # colapsa espaços duplicados
      .str.upper()               # padroniza para maiúsculas
)

# Vamos inspecionar as 20 primeiras colunas para confirmar a padronização:
print("Colunas (amostra):", df.columns.tolist()[:20])
print("Total de colunas:", len(df.columns))
print("Total de linhas:", len(df))

#Checagens iniciais e perfil de nulos
- realizar o primeiro diagnóstico de qualidade dos dados antes de qualquer normalização, clusterização ou modelagem

In [ ]:

# ------------------------------------------------------------------------------------
# PASSO 2: Perfil de nulos e tipo geral dos dados
# - Quantifica nulos por coluna (para orientar limpeza posterior)
# - Mostra tipos inferidos pelo pandas
# ------------------------------------------------------------------------------------

# Contagem de nulos por coluna (ordenada decrescente):
null_count = df.isna().sum().sort_values(ascending=False)
print("Nulos por coluna (top 20):")
print(null_count.head(20))

# Tipos (dtypes) do pandas:
print("\nTipos inferidos pelo pandas:")
print(df.dtypes)

#Remover colunas que não agregam ao dataset analítico inicial

In [ ]:

cols_drop_safe = [
    "OID",
    "PIPERUN_OID",
    "PIPERUN_TAG",
    "DESIGN_STATUS",
    "VALVEWIZARD_SELECTED",
    "VALVEWIZARD_INPUT",
    "MECHANICS_REMARKS",
    "PIPELINE_STATUS",
    "COMPONENT_STATUS"
]

# Filtra apenas as colunas que existem de fato no df (para evitar KeyError):
exist_safe = [c for c in cols_drop_safe if c in df.columns]

# Remoção:
df = df.drop(columns=exist_safe)

print(f"Removidas (safe): {exist_safe}")
print("Colunas restantes:", len(df.columns))

#Normalização do diâmetro nominal
- Converte representações como 1-1/2", 3/4", 10" para float em polegadas (NPS)
- Cria flag NPS_FLAG com faixa plausível (ajuste conforme necessidade)


In [ ]:

import re
import numpy as np
import pandas as pd

def parse_nps(value):
    """
    Converte '1-1/2"', '3/4"', '10"', etc. em float (polegadas).
    Retorna np.nan se não conseguir interpretar.
    """
    if pd.isna(value):
        return np.nan
    s = str(value).strip()
    s = s.replace('"', '').replace("'", '').replace(',', '.')
    # padrão 'a-b/c'
    m = re.match(r'^(\d+)-(\d+)/(\d+)$', s)
    if m:
        a, b, c = m.groups()
        try:
            return float(a) + float(b)/float(c)
        except Exception:
            return np.nan
    # limpar mantendo dígitos, ponto, barra e hífen
    s2 = re.sub(r'[^0-9\./-]', '', s)
    # fração com hífen 'a-b/c'
    if '/' in s2 and '-' in s2:
        try:
            base, frac = s2.split('-')
            num, den = frac.split('/')
            return float(base) + float(num)/float(den)
        except Exception:
            return np.nan
    # fração simples 'b/c'
    if '/' in s2 and '-' not in s2:
        try:
            num, den = s2.split('/')
            return float(num)/float(den)
        except Exception:
            return np.nan
    # número simples
    try:
        return float(s2)
    except Exception:
        return np.nan

if "VALV_DIAMETER_RAW" in df.columns:
    df["NPS_IN"] = df["VALV_DIAMETER_RAW"].apply(parse_nps)
elif "VALV_DIAMETER" in df.columns:
    df["NPS_IN"] = df["VALV_DIAMETER"].apply(parse_nps)

# Faixa plausível (ajuste conforme o padrão de projeto): 0.25" a 60"
if "NPS_IN" in df.columns:
    df["NPS_FLAG"] = np.where(df["NPS_IN"].between(0.25, 60, inclusive="both"),
                              "OK", "OUT_OF_RANGE")

#Normalização de temperaturas para float (°C) e coerência de faixa
- Converte DESIGN_MIN_TEMP, OPER_NORM_TEMP, DESIGN_MAX_TEMP em *_NORM (float)
- TEMP_RANGE_FLAG: 'OK' se min <= oper <= max; 'INCONSISTENT' caso contrário
- TEMP_OUTLIER_FLAG: marca outliers amplos (ex.: min < -200°C ou max > 500°C)

In [ ]:

def to_float(series):
    return pd.to_numeric(series, errors="coerce")

for c in ["DESIGN_MIN_TEMP", "OPER_NORM_TEMP", "DESIGN_MAX_TEMP"]:
    src = f"{c}_RAW" if f"{c}_RAW" in df.columns else c
    if src in df.columns:
        df[f"{c}_NORM"] = to_float(df[src])

# Coerência: min <= oper <= max (se todos existirem)
required = ["DESIGN_MIN_TEMP_NORM","OPER_NORM_TEMP_NORM","DESIGN_MAX_TEMP_NORM"]
if all(col in df.columns for col in required):
    cond_ok = (
        (df["DESIGN_MIN_TEMP_NORM"].isna() |
         df["OPER_NORM_TEMP_NORM"].isna() |
         df["DESIGN_MAX_TEMP_NORM"].isna())
        |
        ((df["DESIGN_MIN_TEMP_NORM"] <= df["OPER_NORM_TEMP_NORM"]) &
         (df["OPER_NORM_TEMP_NORM"] <= df["DESIGN_MAX_TEMP_NORM"]))
    )
    df["TEMP_RANGE_FLAG"] = np.where(cond_ok, "OK", "INCONSISTENT")

    # Outlier simples (ajuste conforme necessidade de engenharia)
    df["TEMP_OUTLIER_FLAG"] = np.where(
        (df["DESIGN_MIN_TEMP_NORM"] < -200) | (df["DESIGN_MAX_TEMP_NORM"] > 500),
        "POTENTIAL_OUTLIER", "OK"
    )

#Normalização da pressão máxima de projeto
- Converte para bar em DESIGN_MAX_PRESS_BAR
- Heurística: se valor >= 50.000, tratamos como Pa e dividimos por 100.000
- PRESS_FLAG: 'OK' se 0.1 <= bar <= 1000; 'OUT_OF_RANGE' caso contrário

In [ ]:
def to_bar(x):
    val = pd.to_numeric(x, errors="coerce")
    if pd.isna(val):
        return np.nan
    # Heurística: valores grandes provavelmente em Pascal
    return val / 1e5 if val >= 5e4 else val  # 100.000 Pa = 1 bar

src_press = "DESIGN_MAX_PRESS_RAW" if "DESIGN_MAX_PRESS_RAW" in df.columns else "DESIGN_MAX_PRESS"
if src_press in df.columns:
    df["DESIGN_MAX_PRESS_BAR"] = df[src_press].apply(to_bar)
    df["PRESS_FLAG"] = np.where(df["DESIGN_MAX_PRESS_BAR"].between(0.1, 1000, inclusive="both"),
                                "OK", "OUT_OF_RANGE")

#Normalização de vocabulário
- TYPE_NORM: consolida sinônimos (ex.: 'BALL VALVE' -> 'BALL')
- VALVE_APP_NORM: padroniza grafia e categorias
- FLUID_CODE_NORM: padroniza para maiúsculas, sem espaços

In [ ]:
# Mapeamento de TYPE (ajuste conforme sua convenção interna)
type_map = {
    "BALL VALVE": "BALL",
    "BALL": "BALL",
    "GLOBE VALVE": "GLOBE",
    "GLOBE": "GLOBE",
    "CHECK": "CHECK",
    "NON-SLAM CHECK": "CHECK",
    "GATE": "GATE",
    "RELIEF VALVE PRESSURE": "PSV",
    "BI OFFSET BUTTERFLY": "BUTTERFLY (DOUBLE OFFSET)",
    "CONC BUTTERFLY": "BUTTERFLY (CONCENTRIC)",
    "NEEDLE": "NEEDLE",
    "DOUBLE BALL": "DOUBLE BALL",
}
if "TYPE" in df.columns:
    df["TYPE_NORM"] = (
        df["TYPE"].astype(str).str.strip().str.upper()
        .map(lambda x: type_map.get(x, x))
    )

# Mapeamento de VALVE_APPLICATION (exemplo; ajuste conforme seu padrão)
app_map = {
    "PSV BLOCK OUTLET": "PSV BLOCK OUTLET",
    "PSV BLOCK INLET": "PSV BLOCK INLET",
    "PSV": "PSV",
    "CHECK VALVE": "CHECK",
    "XV": "XV",
    "SDV": "SDV",
    "BDV BLOCK": "BDV BLOCK",
    "INSTRUMENT BLOCK": "INSTRUMENT BLOCK",
    "LIT / LG BLOCK": "LEVEL BLOCK",
    "DRAIN / UTILITY": "DRAIN/UTILITY",
    "OTHER BLOCK": "OTHER BLOCK",
    "MANUAL CONTROL": "MANUAL CONTROL",
}
if "VALVE_APPLICATION" in df.columns:
    df["VALVE_APP_NORM"] = (
        df["VALVE_APPLICATION"].astype(str).str.strip().str.upper()
        .map(lambda x: app_map.get(x, x))
    )

# FLUID_CODE → maiúsculas, sem espaços
if "FLUID_CODE" in df.columns:
    df["FLUID_CODE_NORM"] = df["FLUID_CODE"].astype(str).str.strip().str.upper()

#Completude (campos críticos) e chave técnica composta
- MISSING_FLAG: marca registros com ao menos 1 campo crítico nulo
- KEY_TECH: concatena MODULE|TAG|PLANTGROUP para rastreabilidade técnica

In [ ]:

critical_for_completeness = [
    "TYPE","VALVE_APPLICATION","FLUID_CODE","VALV_DIAMETER",
    "DESIGN_MAX_TEMP","DESIGN_MIN_TEMP","DESIGN_MAX_PRESS",
    "TAG","MODULE","PLANTGROUP"
]
critical_present = [c for c in critical_for_completeness if c in df.columns]

if critical_present:
    df["MISSING_FLAG"] = np.where(df[critical_present].isna().any(axis=1), "HAS_MISSING", "OK")

# Garantir existência das colunas de chave
for c in ["MODULE","TAG","PLANTGROUP"]:
    if c not in df.columns:
        df[c] = np.nan

df["KEY_TECH"] = (
    df["MODULE"].astype(str).str.strip() + "|" +
    df["TAG"].astype(str).str.strip() + "|" +
    df["PLANTGROUP"].astype(str).str.strip()
)

# CLUSTERIZAÇÃO
- Seleção de features e preparação (numéricos + categóricos)

- Numéricos: NPS_IN, DESIGN_MAX_PRESS_BAR, DESIGN_MIN_TEMP_NORM, OPER_NORM_TEMP_NORM, DESIGN_MAX_TEMP_NORM
- Categóricos (codificação one‑hot): TYPE_NORM, VALVE_APP_NORM, FLUID_CODE_NORM
- Removidas linhas com nulos apenas nas colunas de features para evitar quebra do pipeline.

In [ ]:



# Escolha das colunas de entrada (ajuste conforme necessário):
num_features = [
    "NPS_IN",
    "DESIGN_MAX_PRESS_BAR",
    "DESIGN_MIN_TEMP_NORM",
    "OPER_NORM_TEMP_NORM",
    "DESIGN_MAX_TEMP_NORM",
]
cat_features = [
    "TYPE_NORM",
    "VALVE_APP_NORM",
    "FLUID_CODE_NORM",
]

# Filtra o DataFrame para conter somente as colunas de interesse
available_num = [c for c in num_features if c in df.columns]
available_cat = [c for c in cat_features if c in df.columns]
X = df[available_num + available_cat].copy()

# Remover linhas com nulos nas features (mínimo necessário para scikit-learn)
X = X.dropna(subset=available_num)  # categóricas podem ter NaN, vamos preenchê-las no encoder
print(f"Registros disponíveis para clusterização: {len(X)}")

# Mantém índice original para mapear clusters de volta ao df principal
idx_map = X.index

# CLUSTERIZAÇÃO PASSO 2:
- ColumnTransformer + Pipeline
- Numéricos → StandardScaler (média=0, desvio=1).
- Categóricos → OneHotEncoder (gera colunas binárias; usamos handle_unknown='ignore').

In [ ]:
# ------------------------------------------------------------------------------------

# ------------------------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Transformadores
numeric_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# ColumnTransformer: combina ambos
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, available_num),
        ("cat", categorical_transformer, available_cat),
    ],
    remainder="drop"
)

#Baseline com K‑Means (escolha de k usando silhouette)

- Testei alguns valores de k e escolhi aquele com melhor silhouette score.
- Depois rodei K‑Means com o k escolhido e armazenei os rótulos no df.

In [ ]:

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Lista de k para testar (ajuste conforme necessidade)
candidate_k = [4, 5, 6, 7, 8]

# Pré-processa os dados (escala + one-hot)
X_transformed = preprocess.fit_transform(X)

best_k = None
best_score = -1
scores = {}

for k in candidate_k:
    # n_init moderno usa 'auto' (sklearn>=1.4); se a versão for anterior, use n_init=10
    km = KMeans(n_clusters=k, random_state=42, n_init="auto")
    labels = km.fit_predict(X_transformed)
    # Evitar silhouette com 1 cluster
    score = silhouette_score(X_transformed, labels) if len(set(labels)) > 1 else -1
    scores[k] = score
    if score > best_score:
        best_score = score
        best_k = k

print("Scores de silhouette por k:", scores)
print("Melhor k:", best_k, " | silhouette:", round(best_score, 4))

# Treina K-Means final com o melhor k
km_final = KMeans(n_clusters=best_k, random_state=42, n_init="auto")
labels_km = km_final.fit_predict(X_transformed)

# Persiste rótulos no df original
df.loc[idx_map, "CLUSTER_KMEANS"] = labels_km

# Centroides (no espaço transformado); para interpretação, podemos usar médias por cluster no espaço original
centroids_transformed = km_final.cluster_centers_
print("K-Means concluído. Clusters atribuídos em 'CLUSTER_KMEANS'.")

#Interpretação prática dos clusters - cluster_kmeans_summary.csv
- Cálculo de médias e proporções das variáveis no espaço original
- planilha gerada contém resumo por cluster do K‑Means, mostrando contagem, médias, mínimos e máximos das variáveis numéricas e os TOP valores categóricos (TYPE, FLUID).
- Útil para interpretar cada cluster como um regime técnico (perfil de válvulas, condições de processo).

In [ ]:

cluster_summary = (
    df.loc[idx_map, ["CLUSTER_KMEANS"] + available_num + available_cat]
      .groupby("CLUSTER_KMEANS")
      .agg({
          **{c: ["count", "mean", "min", "max"] for c in available_num},
          **{c: lambda s: s.value_counts().head(3) for c in available_cat}
      })
)

print(cluster_summary)
# Opcional: salvar para consulta
cluster_summary.to_csv("cluster_kmeans_summary.csv")

#DBSCAN (densidade + outliers)

In [ ]:

from sklearn.cluster import DBSCAN

# Pré-processamento (já feito acima): X_transformed
# Parâmetros iniciais (ajuste conforme inspeção dos resultados):
db = DBSCAN(eps=1.5, min_samples=40, n_jobs=-1)
#eps = 1.2 a 2.0 e min_samples = 30 a 60 (valores ótimos)
labels_db = db.fit_predict(X_transformed)

# -1 indica outliers (ruído) no DBSCAN
df.loc[idx_map, "CLUSTER_DBSCAN"] = labels_db

# Quantitativo de clusters e outliers:
unique_labels = pd.Series(labels_db).value_counts().sort_index()
print("Clusters DBSCAN (inclui -1 = outliers):")
print(unique_labels)

# Percentual de outliers:
pct_outliers = (unique_labels.get(-1, 0) / len(labels_db)) * 100
print(f"Outliers (DBSCAN): {pct_outliers:.2f}%")

#Sumário dos grupos DBSCAN (excluindo outliers) - cluster_dbscan_summary.csv
- Exclui o rótulo -1 (outliers) para sumarização
- planilha gerada contém um resumo dos clusters densos encontrados pelo DBSCAN (exclui outliers), com estatísticas numéricas e TOP categorias por cluster.
- Ajuda a entender regimes reais de operação e diferenciar grupos coerentes dos outliers detectados pelo modelo.

In [ ]:

valid_mask = labels_db != -1
dbscan_summary = (
    df.loc[idx_map[valid_mask], ["CLUSTER_DBSCAN"] + available_num + available_cat]
      .groupby("CLUSTER_DBSCAN")
      .agg({
          **{c: ["count", "mean", "min", "max"] for c in available_num},
          **{c: lambda s: s.value_counts().head(3) for c in available_cat}
      })
)

print(dbscan_summary)
dbscan_summary.to_csv("cluster_dbscan_summary.csv")

#Exportar resultados (rótulos + perfis) - valves_clusters.csv
dataset final do estudo, já normalizado, contendo:

- Todos os dados tratados (NPS em polegadas, pressão em bar, temperaturas em °C);
- As features usadas no modelo (numéricas + categóricas normalizadas);
- Os rótulos dos modelos (CLUSTER_KMEANS e CLUSTER_DBSCAN);
- As flags de qualidade (outliers, incoerências, nulos, faixas inválidas);
- As colunas RAW vs NORMALIZADAS para rastreabilidade.

In [ ]:

# DataFrame com rótulos
out = df.loc[idx_map, ["TAG","MODULE","PLANTGROUP"] + available_num + available_cat + ["CLUSTER_KMEANS","CLUSTER_DBSCAN"]]
out.to_csv("valves_clusters.csv", index=False)
print("Arquivo gerado: valves_clusters.csv (rótulos K-Means e DBSCAN).")

#Geração de tabela PCA (PC1 × PC2) para auxiliar na análise gráfica de múltiplas variáveis originais

In [ ]:
from sklearn.decomposition import PCA

# Carregar a base
df = pd.read_csv("valves_clusters.csv")

# Selecionar somente variáveis numéricas usadas no clustering
cols_num = [
    "NPS_IN",
    "DESIGN_MAX_PRESS_BAR",
    "DESIGN_MIN_TEMP_NORM",
    "OPER_NORM_TEMP_NORM",
    "DESIGN_MAX_TEMP_NORM"
]

X = df[cols_num]

# Normalização (igual ao pipeline do clustering)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# PCA — reduzindo para 2 dimensões
pca = PCA(n_components=2)
pcs = pca.fit_transform(X_scaled)

df["PC1"] = pcs[:, 0]
df["PC2"] = pcs[:, 1]
#df["CLUSTER_DBSCAN"] = (df["CLUSTER_DBSCAN"] / 10).astype(int)

# Mantém os clusters DBSCAN e KMeans
df_pca = df[[
    "TAG",
    "PC1","PC2",
    "CLUSTER_KMEANS",
    "CLUSTER_DBSCAN",
    "TYPE_NORM",
    "VALVE_APP_NORM",
    "FLUID_CODE_NORM"
]]
#df["CLUSTER_DBSCAN"] = (df["CLUSTER_DBSCAN"] / 10).astype(int)
df_pca.to_csv("pca_coords.csv", index=False)
print("Arquivo gerado: pca_coords.csv")

#Visualizações gráficas dos resultados

#**K-means Raciocínio**:
O cluster Kmeans não identifica outliers, ele força cada ponto a pertencer a um cluster, mesmo que seja um ponto completamente anormal, porém, visualmente, é possível identificar de forma clara valores distantes que se comportam como outliers




In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 8))
sns.scatterplot(
    x='PC1',
    y='PC2',
    hue='CLUSTER_KMEANS',
    data=df_pca_coords,
    palette='viridis',
    legend='full'
)
plt.title('Clusters K-Means no Espaço PCA')
plt.xlabel('Componente Principal 1 (PC1)')
plt.ylabel('Componente Principal 2 (PC2)')
plt.grid(True)
plt.show()

#**DBSCAN Raciocínio:**

O cluster DBSCAN separa os outliers (valor -1) pela cor azul escuro. O motivo pelo qual aparecem outliers dentro das nuvens de clusters é porque o PCA é apenas uma representação em duas dimensões de um espaço multidimensional. Sendo assim, o PCA “achata” o espaço e pode projetar um ponto distante em cima da nuvem

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x='PC1',
    y='PC2',
    hue='CLUSTER_DBSCAN',
    data=df_pca_coords,
    palette='tab20', # Use a different palette for DBSCAN to distinguish from K-Means
    legend='full'
)
plt.title('Clusters DBSCAN no Espaço PCA')
plt.xlabel('Componente Principal 1 (PC1)')
plt.ylabel('Componente Principal 2 (PC2)')
plt.grid(True)
plt.show()

**Raciocínio**:

Os gráficos a seguir mostram a distribuição dos clusters pelas categorias "Tipos de válvulas", "Aplicação" e "Tipos de fluidos"



In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x='PC1',
    y='PC2',
    hue='TYPE_NORM',
    data=df_pca_coords,
    palette='tab20',
    legend='full'
)
plt.title('Distribuição por tipo de válvula no espaço PCA')
plt.xlabel('Componente Principal 1 (PC1)')
plt.ylabel('Componente Principal 2 (PC2)')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
sns.scatterplot(
    x='PC1',
    y='PC2',
    hue='VALVE_APP_NORM',
    data=df_pca_coords,
    palette='tab20',
    legend='full'
)
plt.title('Distribuição por aplicação de válvulas no espaço PCA')
plt.xlabel('componente principal 1 (PC1)')
plt.ylabel('componente principal 2 (PC2)')
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(12, 10))
sns.scatterplot(
    x='PC1',
    y='PC2',
    hue='FLUID_CODE_NORM',
    data=df_pca_coords,
    palette='tab20',
    legend='full'
)
plt.title('Distribuição por tipo de fluido no espaço PCA')
plt.xlabel('componente principal 1 (PC1)')
plt.ylabel('componente principal 2 (PC2)')
plt.grid(True)
plt.show()

# ------------------------ FIM ---------------------------------